
# List of Packages to Download

```python
!pip install langchain-chroma
!pip install langchain
!pip install langchain_community
!pip install langchainhub
!pip install -qU langchain-openai

```
## Also fix api key


```python

import os

os.environ["OPENAI_API_KEY"] = "your-openai-key"
```


______________________________________________________________________________
 

# 1. Get a Data Loader


In [16]:
from langchain_community.document_loaders import WebBaseLoader


In [17]:
loader = WebBaseLoader("https://medium.com/@daniellefranca96/gpt4-omni-so-much-more-than-just-a-voice-assistant-c5ae43bdc6be")
data = loader.load()
data

[Document(page_content='GPT4 Omni — So much more than just a voice assistant | by katerinaptrv | May, 2024 | MediumOpen in appSign upSign inWriteSign upSign inGPT4 Omni — So much more than just a voice assistantkaterinaptrv·Follow3 min read·1 hour ago--ListenShareToday we have the OpenAI Spring announcement and it was mind blowing, I guess we all can agree here. I was playing with my new voice assistant for most of this night(mentions of the movie HER very accurate in this scenario).But as revolutionary, incredible and world changing as the voice capability is, the whole GPT4o model goes so much further than that.I must confess, the voice thing got me for real, so even I took a time to really stop and read the technical announcement of this model and when I finally did just now my mind was blown again.I was thinking GPT4o was just a better optimized version of GPT-4 Turbo, this time with better reasoning, less latency and trained for voice conversations. And that they basically just co

# 2. Convert data to Vector Database


In [18]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


In [19]:
Chroma.from_documents(documents=data, embedding=OpenAIEmbeddings(model="text-embedding-3-large"), persist_directory="./chroma_db")

# 3. Make a RAG pipeline


In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [21]:
llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"))


In [22]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [23]:
question = "Make SEO optimized GPT4o youtube description"
result = qa_chain({"query": question })

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [24]:
result["result"]

'Explore the revolutionary capabilities of GPT4 Omni in our latest video! Discover how this groundbreaking AI model integrates text, vision, and audio processing to deliver unparalleled performance and efficiency. Subscribe now to stay updated on the future of AI technology! #GPT4Omni #AI #MachineLearning #DeepLearning #OpenAI'

In [25]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

('Explore the revolutionary capabilities of GPT4 Omni in our latest video! '
 'Discover how this groundbreaking AI model integrates text, vision, and audio '
 'processing to deliver unparalleled performance and efficiency. Subscribe now '
 'to stay updated on the future of AI technology! #GPT4Omni #AI '
 '#MachineLearning #DeepLearning #OpenAI')
